In [7]:
import pandas as pd
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
df_requests = pd.read_excel('FRC Data1.xlsx', sheet_name='FRC Requests')
df_people = pd.read_excel('FRC Data1.xlsx', sheet_name='Requestor')

In [35]:
df_requests['Category'].value_counts()

Process    3727
Systems    1859
Forms      1458
Rate       1140
Other       457
Name: Category, dtype: int64

In [24]:
df_requests['Key Word Roll-Up'].value_counts()

Rate Change                 556
Form Review                 262
Endorsement Print           250
Premium Discrepancy         218
CAT                         194
Endorsement Process         186
Experience Modification     181
Billing                     166
System Issue                165
Policy Setup                157
Rating Error                152
BOR                         135
Policy Issuance             130
Rating Process              125
Officer Exclusion           123
Loss Analysis               123
LOB Assistance              121
Premium Issue               115
RE Factor                   114
Technical Error             113
Filing                      110
Audit                       105
User Access                 103
Classification              100
Technical Rating             94
Form Missing                 90
Policy Print                 89
Cancel Rewrite               87
Composite                    85
Endorsement Correction       85
                           ... 
CNA Emer

In [5]:
df_requests.head()

,ID,Submission Date,Resolution Date,FRC Owner,Request,Requestor,Request Description,Status,Urgent?,Reopen?,...,Resource(s) Used / Referred To,Incident / PME / Project # (Where Appl.),Created,Created By,Modified,Last Modified By,Submission Month,res date,Item Type,Path
0,13960,2018-10-01,NaT,"Bush, Andrew",return premium endorsement was booked in Rapid,"Lal,Pushpanjali","\n​\nGood Morning All,\n \nAs I mentioned belo...",Open,No,No,...,NaN,NaN,2018-10-01 10:44:16,"Munsell,Julia E.",2018-10-01 10:44:16,"Munsell,Julia E.",201810,-,Item,ccm/FRC/Lists/TEST
1,13959,2018-10-01,NaT,"Reel, Joshua",CRIS training ?,"Albright,Kevin T.",\n​Please advise the path for an employee to r...,Open,No,No,...,NaN,NaN,2018-10-01 10:33:36,"Munsell,Julia E.",2018-10-01 10:33:36,"Munsell,Julia E.",201810,-,Item,ccm/FRC/Lists/TEST
2,13958,2018-10-01,NaT,"Brooks, Jayson",reference connect,"Amato,Marisol",\n​\nHello\n \nI need to get a couple of publi...,Open,No,No,...,NaN,NaN,2018-10-01 10:32:12,"Munsell,Julia E.",2018-10-01 10:32:12,"Munsell,Julia E.",201810,-,Item,ccm/FRC/Lists/TEST
3,13957,2018-10-01,NaT,"Bush, Andrew",Tech Rating,"Miletich,Denise",\n​Tech RAting error,Pending Follow-Up Outside FRC,No,No,...,TAP IT,INC0692683,2018-10-01 10:29:46,"Bush,Andrew",2018-10-01 10:29:46,"Bush,Andrew",201810,-,Item,ccm/FRC/Lists/TEST
4,13956,2018-10-01,NaT,"Ewing, Jamie",Submission Documents in EPC,"Howard,Anne",\n​\nI'm not able to open new business submiss...,Open,No,No,...,NaN,NaN,2018-10-01 09:40:43,"Munsell,Julia E.",2018-10-01 09:40:43,"Munsell,Julia E.",201810,-,Item,ccm/FRC/Lists/TEST


In [6]:
df_people.head()

,Full Name,Business Unit Name,Dept Name,Line of Business,Job Title,Current Manager Name,Location - Name
0,"Pollard,Jacqueline D",Worldwide Field Operations,Field Ops S-Atlanta Ops-2,UW-Comm-Middle Market,Sr. Underwriting Technician,"Mauney, James Robert",US- GA71- Atlanta-5565 Glenrdg
1,"Antkowiak,Jon P",Claim,CLM-ClmLaw-LglSrvcsE2-HART,NaN,Senior Litigation Attorney,"Jaworski, Cynthia A",US- CT72 - Glastonbry-Blvd
2,"Gagne,Janice E",Worldwide Field Operations,Field Ops S-Atlanta Ops-3,UW-Comm-Middle Market,Sr. Underwriting Technician,"McCann, Karen L.",US- FL84 -Lake Mary-500 Col
3,"Lord,Charles S",Commercial Insurance,Comm Ins-UW-LOB-Packageincl GL,UW-Comm-LOB-GL,Underwriting Consltg Dir,"Walsh, Kelly",US- IL40- Chicago-151N Frankln
4,"LeMay,Lorna J",Worldwide Field Operations,Field Ops-NE-TM 3-1,UW-Comm-Middle Market,Underwriting Technician,"Wojtkow, Carol Anne",US- MA17 -Boston-53 State St


In [8]:
train_size = int(len(df_requests) * .8)
train_posts = df_requests['Request Description'][:train_size]
train_tags = df_requests['Key Word Roll-Up'][:train_size]
test_posts = df_requests['Request Description'][train_size:]
test_tags = df_requests['Key Word Roll-Up'][train_size:]

In [27]:
vocab_size = 10000
tokenize = text.Tokenizer(num_words=vocab_size)
tokenize.fit_on_texts(train_posts)

array([[0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.]])

In [28]:
x_train = tokenize.texts_to_matrix(train_posts)

In [29]:
encoder = LabelBinarizer()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

In [30]:
model = Sequential()

In [31]:
model.add(Dense(512, input_shape=(vocab_size,)))
model.add(Activation('relu'))

In [32]:
model.add(Dense(y_train.shape[1]))
model.add(Activation('softmax'))

In [33]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [34]:
history = model.fit(x_train, y_train, 
                    batch_size=100, 
                    epochs=4, 
                    verbose=1, 
                    validation_split=0.1)

Train on 6221 samples, validate on 692 samples
Epoch 1/4
6221/6221 [==============================] - 76s 12ms/step - loss: 4.9776 - acc: 0.1628 - val_loss: 4.5933 - val_acc: 0.1575
Epoch 2/4
6221/6221 [==============================] - 77s 12ms/step - loss: 3.3108 - acc: 0.4374 - val_loss: 4.0766 - val_acc: 0.2601
Epoch 3/4
6221/6221 [==============================] - 80s 13ms/step - loss: 2.2053 - acc: 0.6343 - val_loss: 3.7800 - val_acc: 0.2991
Epoch 4/4
6221/6221 [==============================] - 80s 13ms/step - loss: 1.4466 - acc: 0.7701 - val_loss: 3.6414 - val_acc: 0.3165


In [36]:
df_list = df_requests['Key Word Roll-Up'].value_counts()

In [37]:
writer = pd.ExcelWriter('categories_output.xlsx')
df_list.to_excel(writer,'Sheet1')
writer.save()

In [40]:
key_word_list = ['Rate Change',
'Form Review',
'Endorsement Print',
'Premium Discrepancy',
'CAT',
'Endorsement Process',
'Experience Modification',
'Billing',
'System Issue',
'Policy Setup',
'Rating Error',
'BOR',
'Policy Issuance',
'Rating Process',
'Officer Exclusion',
'Loss Analysis',
'LOB Assistance'
'Premium Issue']
condensed_list = []
for row in df_requests['Key Word Roll-Up']:
    if row in key_word_list:
        condensed_list.append(row)
    else: 
        condensed_list.append('Other')

In [42]:
len(condensed_list)

8642

In [43]:
condensed_list

['Endorsement Process',
 'Other',
 'Other',
 'Other',
 'System Issue',
 'Other',
 'Other',
 'Other',
 'Other',
 'Other',
 'Other',
 'Other',
 'Policy Setup',
 'Other',
 'Experience Modification',
 'Form Review',
 'Other',
 'Other',
 'Officer Exclusion',
 'Other',
 'Billing',
 'Officer Exclusion',
 'Endorsement Print',
 'Policy Setup',
 'Policy Issuance',
 'Rate Change',
 'Other',
 'Policy Issuance',
 'Other',
 'Billing',
 'Policy Issuance',
 'Rate Change',
 'Endorsement Process',
 'Rate Change',
 'Other',
 'Other',
 'Other',
 'Other',
 'Policy Issuance',
 'Rating Error',
 'Endorsement Print',
 'Other',
 'Other',
 'System Issue',
 'Rate Change',
 'Policy Issuance',
 'Other',
 'Other',
 'Other',
 'Policy Issuance',
 'Billing',
 'Other',
 'Other',
 'Other',
 'Other',
 'Premium Discrepancy',
 'Premium Discrepancy',
 'Other',
 'Policy Setup',
 'BOR',
 'Rate Change',
 'Form Review',
 'Rate Change',
 'Other',
 'Premium Discrepancy',
 'Premium Discrepancy',
 'Premium Discrepancy',
 'Premium Di